In [5]:
import requests
import pandas as pd

# Step 1: Search for restaurants to get place IDs
def search_restaurants(api_key, location):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": location,
        "radius": 16360,
        "type": "restaurant",
        "key": api_key
    }
    
    response = requests.get(base_url, params=params)
    places = response.json().get("results", [])
    
    return [place["place_id"] for place in places]

# Step 2: Fetch place details for each place ID to get reviews
def fetch_place_details(api_key, place_id):
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "name,reviews",
        "key": api_key
    }
    
    response = requests.get(base_url, params=params)
    return response.json().get("result", {})

def main(api_key, location):
    place_ids = search_restaurants(api_key, location)
    all_reviews = []

    for place_id in place_ids:
        details = fetch_place_details(api_key, place_id)
        name = details.get("name", "")
        
        reviews = details.get("reviews", [])
        for review in reviews:
            all_reviews.append({
                "name": name,
                "rating": review.get("rating"),
                "text": review.get("text")
            })

    df = pd.DataFrame(all_reviews)
    df.to_excel("restaurant_reviews_daegu.xlsx", index=False, engine="openpyxl")



# Example usage
api_key = "YOUR_API_KEY"
location = "37.6, 127"  # Example: 서울 37.6, 127
main(api_key, location)
